In [1]:
# Impordid
from src.transformers.models.bert.tokenization_bert import BertTokenizer
from src.transformers.models.bert.modeling_bert import BertEmbeddings, BertModel, BertForMaskedLM
import torch
from estnltk import Text
from estnltk.corpus_processing.parse_enc import parse_enc_file_iterator
from src.transformers.models.bert.configuration_bert import BertConfig
from transformers import AdamW
from tqdm.auto import tqdm
from transformers import pipeline
import numpy as np

In [2]:
# Tokeniseerija
tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt")

In [3]:
%%time
# Korpus
# Loeme tekstid sisse, laseme estnltk-l laused leida ning moodustame lausetest treening- ja testhulg
# https://github.com/estnltk/estnltk/blob/main/tutorials/corpus_processing/importing_text_objects_from_corpora.ipynb
input_file = "korpus/estonian_nc17.vert.01"
n = 2 # Mitu teksti korpusesse lugeda
korpus = []
l = 0
for text_obj in parse_enc_file_iterator(input_file):
    korpus.append(text_obj)
    if l > n:
        break
    l += 1

train =  korpus[:int(0.8*len(korpus))]
test = korpus[int(0.8*len(korpus)):]

train_laused = []
for tekst in train:
    for span in tekst.original_sentences:
        train_laused.append(tekst.text[span.start:span.end])
        
test_laused = []
for tekst in test:
    for span in tekst.original_sentences:
        test_laused.append(tekst.text[span.start:span.end])

CPU times: total: 344 ms
Wall time: 349 ms


In [4]:
# Dataseti loomine dataloaderi jaoks
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings["input_ids"].shape[0]
    def __getitem__(self, i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [5]:
# Maskimine ja labelid
def mlm(tensor):
    labels = tensor.detach().clone()
    rand = torch.rand(tensor[:, :, 0].shape)
    mask_arr = (rand < 0.15) * (tensor[:, :, 0] > 5)
    for i in range(tensor[:, :, 0].shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        selection_masked = torch.where(mask_arr[i] == 0)[0].tolist()
        tensor[i, selection] = 4 # [MASK] tokeni ID mõlemas vocabis
        labels[i, selection_masked] = -100 # Et mudel arvutaks lossi ainult masked tokenite pealt
        
    return tensor, labels

In [6]:
# Andmestiku ettevalmistamine treenimiseks või mudelis kasutamiseks
def prepare_data(data):
    input_ids = []
    mask = []
    labels = []
    
    sample = tokenizer(data, max_length = 8, padding = "max_length", truncation = True, return_tensors = "pt")
    new_ids, new_labels = mlm(sample.input_ids.detach().clone())
    input_ids.append(new_ids)
    mask.append(sample.attention_mask)
    labels.append(new_labels)
    
    input_ids = torch.cat(input_ids)
    mask = torch.cat(mask)
    labels = torch.cat(labels)
    
    encodings = {
    "input_ids" : input_ids,
    "attention_mask" : mask,
    "labels" : labels
    }
    
    dataset = Dataset(encodings)
    return dataset


# Dataloaderi loomine mudeli treenimiseks
dataloader_train = torch.utils.data.DataLoader(prepare_data(train_laused), batch_size = 16, shuffle = True)

In [9]:
%%time
# Mudeli treenimine

config = BertConfig(
    vocab_size = tokenizer.vocab_size,
    vocab_size_form = tokenizer.vocab_size_form,
    tie_word_embeddings = False
)

model = BertForMaskedLM(config)
linear = torch.nn.Linear(tokenizer.vocab_size, tokenizer.vocab_size_form)
CE = torch.nn.CrossEntropyLoss()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)
optim = AdamW(model.parameters(), lr = 1e-4)
epochs = 100

for epoch in range(epochs):
    loop = tqdm(dataloader_train, leave = True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch["input_ids"]
        mask = batch["attention_mask"]
        labels= batch["labels"]
        output = model(input_ids, attention_mask = mask, labels = labels)
        loss = output.loss # modelling_bert.py liidab lossid juba kokku, niiet see on lemma loss + vormi loss
        loss.backward()
        optim.step()
        loop.set_description(f"Epoch: {epoch}")
        loop.set_postfix(loss = loss.item())

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
# Testimine
dataloader_test = torch.utils.data.DataLoader(prepare_data(test_laused), batch_size = 16)
kokku_labelid_vorm = np.array([-1])
kokku_labelid_lemma = np.array([-1])
kokku_ennustused_vorm = np.array([-1])
kokku_ennustused_lemma = np.array([-1])
#loop2 = tqdm(dataloader_test, leave = True) # Testandmed
loop2 = tqdm(dataloader_train, leave = True) # Treeningandmetel võiks accuracy tulla 1 kui epocheid piisavalt
for batch in loop2:
    input_ids = batch["input_ids"]
    labels = batch["labels"]
    
    output = model(input_ids)
    
    indeksid = np.where(input_ids[:, :, 0].flatten() == 4)
    labels_vorm = labels[:, :, 1].flatten()[indeksid].detach().numpy()
    labels_lemma = labels[:, :, 0].flatten()[indeksid].detach().numpy()
    ennustused_vorm = np.argmax(output.logits[1].flatten(end_dim = 1)[indeksid].detach().numpy(), axis = 1)
    ennustused_lemma = np.argmax(output.logits[0].flatten(end_dim = 1)[indeksid].detach().numpy(), axis = 1)
    
    kokku_labelid_vorm = np.concatenate((kokku_labelid_vorm, labels_vorm))
    kokku_labelid_lemma = np.concatenate((kokku_labelid_lemma, labels_lemma))
    
    kokku_ennustused_vorm = np.concatenate((kokku_ennustused_vorm, ennustused_vorm))
    kokku_ennustused_lemma = np.concatenate((kokku_ennustused_lemma, ennustused_lemma))

  0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
kokku_labelid_vorm
kokku_labelid_lemma

array([   -1,  3045,    98,   101,  1324, 47550,   183,  1545,    77,
          18,  1378,    14,  1571,   313, 27371,   487,    12,  1295,
         245,  2321,   200,     7,   969], dtype=int64)

In [15]:
kokku_ennustused_vorm
kokku_ennustused_lemma

array([   -1,  3045,    98,   101,  1324, 47550,   183,  1545,    77,
          18,  1378,    14,  1571,   313, 27371,   487,    12,  1295,
         245,  2321,   200,     7,   969], dtype=int64)

In [17]:
sum(kokku_ennustused_vorm[1:] == kokku_labelid_vorm[1:])/len(kokku_labelid_vorm[1:])
sum(kokku_ennustused_lemma[1:] == kokku_labelid_lemma[1:])/len(kokku_labelid_lemma[1:])

1.0

In [20]:
#torch.save(model, "mudel.pth")